# Explore and cluster the neighborhoods in Toronto

## Bring datafrme from previous question

In [1]:
%store -r df5
df5

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


import essential library

In [8]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('done')

done


## Explore the Neighbourhood

### Focusing the explore in Borough thats contain the word 'Toronto'

Create the dataframe that Borough contain the word 'Toronto'

In [2]:
Toronto_data = df5[df5['Borough'].str.find('Toronto') != -1].reset_index(drop=True) #.str.find('Text') will eqau; -1 if string not fond 
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Create the Map of Toronto, Canada

import library to create Map

In [3]:
pip install folium

     |████████████████████████████████| 92kB 4.0MB/s 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import folium
print ('Done')

Done


From wikipidia The geograpical coordinate of Toronto, Canada are N 43.6532, W 79.3832 .

In [5]:
#define coridinate of Toronto
latitude = 43.6532
longitude = -79.3832 

#create map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
map_Toronto

Add marker for each postal code to the map

In [6]:
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Postcode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  

map_Toronto

#### Using the Foursquare API to explore the neighborhoods around

Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'XXX' # My Foursquare ID
CLIENT_SECRET = 'XX' # My Foursquare Secret
VERSION = '20190624' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXX
CLIENT_SECRET:XX


#### Explorer top 20 venues that are in each postal code within a radius of 500 meters.

create the GET request URL to retrive data from Foursquar

In [ ]:
LIMIT = 20 # limit of number of venues from Foursquare

radius = 500 # radius around cordinate unit 'meter' 

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL